In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

if os.getcwd().endswith("notebooks"):
    os.chdir("../..")

from ptseries.algorithms.binary_solvers import BinaryBosonicSolver
from ptseries.common.logger import Logger

from PIL import Image
from math import factorial

from ptseries.common.set_seed import set_seed

set_seed(0)

# Optimising Routes for Travel

In this notebook, we will go over how to use the BBS algorithm to solve the *travelling salesperson problem*. The notebook will cover the following:
- Introduction to the travelling salesperson problem
- Description of how to pose the problem as a binary optimisation task
- Setup of the BBS algorithm to solve a toy example of the problem
- Using the BBS algorithm to solve the problem with real world data (cities around the world)
- Creating a sandbox with random data, and solving the travelling salesperson problem with added constraints

We will assume basic knowledge of how the setup of the BBS algorithm works. If you are still unsure of this, see the `optimisation_intro.ipynb` notebook first.

Using quantum processors to solve travelling salesperson problems is a topic which has been explored previously in research work, and the following notebook is in particular inspired by two such publications: https://arxiv.org/pdf/2406.14252, https://arxiv.org/pdf/2404.05448.

## Introduction to the Travelling Salesperson Problem
The travelling salesperson problem is an optimisation task where we are given a set of locations and we want to find the shortest path which visits all of the locations, and returns to the start. In other words, we need to find the shortest round path visiting all of the locations. In this notebook, we will tackle tasks where we assume that we can travel from any one location to another in a straight line, but the problem can be formulated with additional constraints of, say, two locations not actually being connected etc.

Say we are considering $n$ locations. There are $n!$ possible routes that could be taken around all of the locations. As such, it quickly becomes unreasonable to try to check all of the solutions for the shortest path by brute force: for a relatively small problem with 20 locations we already would need to check $20! = 2432902008176640000$ locations.

We will visualise the problem. Let's get some arbitrary coordinates for 5 locations and plot them on a plane.

In [ ]:
# We set the coordinates for 5 locations
x = [12, 26, 40, 95, 43]
y = [2, 42, 72, 78, 12]

n_locations = len(x)

# Plot the locations
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color="gray", marker="o", s=200)

# Add labels to each point
for i in range(len(x)):
    plt.text(x[i], y[i], str(i), fontsize=12, ha="center", va="center", color="black")

# Hides the axes
plt.axis("off")

plt.show()

With such a simple example, we could quite easily take a guess at what an optimal route would be by hand and we'd probably be right. In this case, a path going 0 1 2 3 4 is the natural choice, and is in fact the shortest path. We draw out this route:

In [ ]:
# Plot the locations
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color="gray", marker="o", s=200)

# Add labels to each point
for i in range(len(x)):
    plt.text(x[i], y[i], str(i), fontsize=12, ha="center", va="center", color="black")

# Stores the shortest path between the points
shortest_path = [0, 1, 2, 3, 4]

# Plots the shortest path
for i in range(len(shortest_path) - 1):
    start_loc = shortest_path[i]
    end_loc = shortest_path[i + 1]

    plt.plot([x[start_loc], x[end_loc]], [y[start_loc], y[end_loc]], color="red")

plt.plot(
    [x[shortest_path[-1]], x[shortest_path[0]]],
    [y[shortest_path[-1]], y[shortest_path[0]]],
    color="red",
)

# Hides the axes
plt.axis("off")

plt.show()

Recall, as mentioned earlier that for the travelling salesperson task we have to visit all of the locations in a round trip. So our choice of path 0 1 2 3 4 implicitly represents us also going back to location 0 after location 4.

## The Travelling Salesperson Problem as a Binary Optimisation Task

Now that we understand what the travelling salesperson problem is, we need to figure out how to pose it as a binary optimisation task. This is non-trivial, there isn't necessarily a natural choice of how to encode a path as a binary string. Say we have $n$ locations we need to visit. A first idea might be to encode a route by setting up a bit string of $n$ size $n$ blocks, and having a $1$ in space $j$ of block $i$ if we visit location $j$ at time $i$. This is of course very inefficient as we need bit strings of size $n^2$, however it does lend itself to a QUBO formulation of the problem.

Instead, we'll consider a maximally efficient encoding of the problem into binary, and use a non-QUBO cost function. Note that each route can be described as a permutation of the numbers from $0$ to $n-1$. There are $n!$ such permutations. So, we can correspond each bit string to one of these routes as long as we can find a function mapping bit strings to permutations. We would need length $\lceil \log_2(n!)\rceil$ bit strings for this encoding. Luckily, such a function does exist, and we define it below:

In [ ]:
# Function to convert a binary string to a permutation of 'n_locations' numbers


def binary_to_permutation(binary_string, n_locations):
    x = binary_to_gray(binary_string)

    nodes = [i + 1 for i in range(n_locations)]
    f = factorial(n_locations)
    x = x % f

    # Stores the permutation
    s = []

    for i in range(n_locations):
        f = f / (n_locations - i)
        k = int(np.floor(x / f))

        s.append(nodes[k])

        del nodes[k]

        x = x - k * f

    s = [x - 1 for x in s]
    return s


# Converts a binary string to GrayCode (a different ordering for binary numbers)
def binary_to_gray(bit_string):
    n = int(bit_string, 2)
    return n ^ (n >> 1)

For example, here if we input a binary string '101' we will get a possible permutation (i.e path through the locations) as an output. We get the permutation '0 2 1 4 3', which corresponds to visiting the 0th location then the 2nd then 1st and so on.

In [ ]:
print(binary_to_permutation("101", 5))

All that remains to do now is to define our cost function. The value we want to minimise is the distance of a path, so we need to define a function that takes a permutation (a candidate path) and calculates how long it is.

In [ ]:
# We first need to store the distances between all of the locations we need to visit
n_locations = 5
road_lengths = [[0 for i in range(n_locations)] for j in range(n_locations)]

for i in range(n_locations):
    for j in range(n_locations):
        # Gets the distance between location i and j
        x_length = abs(x[i] - x[j])
        y_length = abs(y[i] - y[j])

        road_length = np.sqrt(x_length**2 + y_length**2)
        road_lengths[i][j] = road_length


# This function takees a possible path, and calculate its total distance
def path_length_calc(path, road_lengths):
    total_distance = 0

    for i in range(len(path) - 1):
        distance = road_lengths[path[i]][path[i + 1]]
        total_distance += distance

    # Adds the final distance between the end location and start location
    distance = road_lengths[path[-1]][path[0]]
    total_distance += distance

    return total_distance

For example, we can now calculate the distance of the optimal path that we earlier found by hand in a simple example. We'll compare it to the length of a path we choose arbitrarily.

In [ ]:
print("Length of shortest path : ")
print(round(path_length_calc([index - 1 for index in shortest_path], road_lengths), 2))

random_path = [3, 0, 2, 1, 4]
print("Length of randomly chosen path : ")
print(round(path_length_calc(random_path, road_lengths), 2))

## Solving a Toy Travelling Salesperson Problem with the BBS Algorithm

Now we have all of the components we need, we will demonstrate running the BBS to find a solution to this example problem. We define a 'cost function' which tells the algorithm which value it wants to minimise. All this function will do is take a binary string, convert it to a permutation which in turn encodes a path, and then calculates the length of that path.

In [ ]:
def cost_fn_fact(road_lengths, n_locations):
    def cost_fn(bit_string):
        bit_string = bit_string.astype(int)

        # Converts the numpy vector into a string
        bit_str = "".join(map(str, bit_string))

        # Converts the bit string into a permutation
        perm = binary_to_permutation(bit_str, n_locations)

        # Gets the length associated with that path
        path_length = path_length_calc(path=perm, road_lengths=road_lengths)

        return path_length

    return cost_fn

Now we will use the built-in BBS function to run the algorithm for us. As mentioned earlier, as we have 5! possible different permutations, we need at least $\log_2 (5!) \approx 6.906$ bits. This means, our binary encoding for each solution will use 7 bits. In the BBS algorithm, this corresponds to having 4 photons and 7 modes in the PT Series device that we are simulating. This is within the specifications of a PT-1 device!

In [ ]:
logger = Logger(log_dir=None)

# Defines the cost function we'll be using
length_cost_fn = cost_fn_fact(road_lengths, n_locations)

# Number of bits we need to encode all possible permutations
n_bits = int(np.ceil(np.log2(factorial(n_locations))))

# Calls the BBS algorithm
bbs = BinaryBosonicSolver(n_bits, length_cost_fn)

# Trains the binary bosonic solver
bbs.train(learning_rate=1e-2, updates=100, print_frequency=10, logger=logger)
print("Training finished")

opt_sol = np.array(bbs.config_min_encountered)
opt_sol = opt_sol.astype(int)
path = binary_to_permutation("".join(map(str, opt_sol)), n_locations)
print("Optimal path found : ")
print(path)
print("Optimal path length : ")
print(str(round(path_length_calc(path, road_lengths), 2)))

We draw the shortest path that the algorithm found.

In [ ]:
# Plot the locations
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color="gray", marker="o", s=200)

# Add labels to each point
for i in range(len(x)):
    plt.text(x[i], y[i], str(i), fontsize=12, ha="center", va="center", color="black")

# Stores the shortest path between the points that the algorithm found
shortest_path = path

# Plots the shortest path
for i in range(len(shortest_path) - 1):
    start_loc = shortest_path[i]
    end_loc = shortest_path[i + 1]

    plt.plot([x[start_loc], x[end_loc]], [y[start_loc], y[end_loc]], color="red")

plt.plot(
    [x[shortest_path[-1]], x[shortest_path[0]]],
    [y[shortest_path[-1]], y[shortest_path[0]]],
    color="red",
)

# Hides the axes
plt.axis("off")

plt.show()

The algorithm has found a path, and we can see that it is exactly the same as the optimal path we found by hand earlier, just with the location order possibly reversed, and starting from a different location. We've seen how we can encode the problem as a binary optimisation task, and use the BBS algorithm with a simulated PT Series device to find a good solution (in this case, the exact solution).

## A Harder Problem - Cities Around The World

Now we move on to a more complicated example. Say we are working for a business in sales, and need to visit lots of different cities around the world in one round trip. We want to find plane routes around the city that will minimise the distance (i.e. time) that we travel. Note that the way we set up the problem will be exactly the same as in the toy example; we will use the same encoder of binary strings to routes and the same cost function. The only difference will be the dataset we use.

In [ ]:
# Loads a map of the world
image_path = "./tutorial_notebooks/figures/World_Map.png"
img = Image.open(image_path)
img_width, img_height = img.size

# Stores the names of the cities we need to visit
city_names = [
    "Austin",
    "London",
    "Seattle",
    "Krakow",
    "Toronto",
    "Tokyo",
    "Delhi",
    "Windhoek",
    "Melbourne",
    "Buenos Aires",
]

# Stores the locations of the cities
coordinates = [
    [436, 658],
    [990, 503],
    [291, 538],
    [1102, 513],
    [537, 570],
    [1780, 624],
    [1424, 678],
    [1085, 978],
    [1815, 1084],
    [660, 1057],
]

# Create a figure and axis to plot the image
fig, ax = plt.subplots(figsize=(14, 10))
ax.imshow(img)

# Plot circles at each location
for i in range(len(coordinates)):
    (x, y) = coordinates[i]
    circle = plt.Circle((x, y), radius=10, color="red", fill=False, linewidth=2)
    ax.add_patch(circle)

    # Add the city name at each location
    ax.text(x, y, f"{city_names[i]}", color="black", fontsize=12, ha="center", va="top")

# Set limits to the image dimensions
ax.set_xlim(0, img_width)
ax.set_ylim(img_height, 0)
ax.axis("off")

# Stores the x,y coordinates of the locations
x = [a for [a, b] in coordinates]
y = [b for [a, b] in coordinates]

# Display the plot
plt.show()

Again, we calculate the distances between the locations and store these.

In [ ]:
n_locations = len(city_names)
# Stores the lengths of the distances between cities
road_lengths = [[0 for i in range(n_locations)] for j in range(n_locations)]


for i in range(n_locations):
    for j in range(n_locations):
        # Gets the length of the distance between location i and j
        x_length = abs(x[i] - x[j])
        y_length = abs(y[i] - y[j])

        road_length = np.sqrt(x_length**2 + y_length**2)
        road_lengths[i][j] = road_length / 100

Now as we have 10 total locations, we have $10! \approx 5,000,000$ possible different paths we can take and so we need at least $\log_2(10!) \approx 21.8$ length bit strings.  There are actually $10! = 3628800$ different possible routes that could be taken! We can see that as these problems get larger, it quickly becomes infeasible to just check every single possible route.

To run the BBS algorithm, we need to simulate a PT Series device with 11 photons and 22 qumodes. This is well within the specifications of a PT-2 device; we will simulate a PT-2 system with a simple single-loop configuration. Below, we call the same cost function and 'binary to permutation' encoders we defined earlier, and run the optimisation algorithm. Note that, since we are simulating a larger quantum system, this will take a longer time to run.

In [ ]:
logger = Logger(log_dir=None)
length_cost_fn = cost_fn_fact(road_lengths, n_locations)

# n bits we need to encode all possible permutations
n_bits = int(np.ceil(np.log2(factorial(n_locations))))

# Initialising the algorithm
bbs = BinaryBosonicSolver(n_bits, length_cost_fn)

# Training the algorithm
bbs.train(learning_rate=1e-2, updates=400, print_frequency=50, logger=logger)
print("Training finished")

opt_sol = np.array(bbs.config_min_encountered)
opt_sol = opt_sol.astype(int)
path = binary_to_permutation("".join(map(str, opt_sol)), n_locations)


We display how the losses the BBS algorithm achieved change throughout the update steps. We can see that, as the algorithm continues, smaller and smaller losses are being achieved. This means that the bit strings being considered are better and better on average. We kept a logger of the training data, so we can now output a graph showing how the losses evolved during the training process.

In [ ]:
# Plot the values for the loss as we go through update steps
def plot_training(logger):
    x_values_qubo = list(map(int, logger.logs["energy_avg"].keys()))
    energies_avg_qubo = list(logger.logs["energy_avg"].values())
    plt.plot(x_values_qubo, energies_avg_qubo)
    plt.xlabel("Updates")
    plt.ylabel("Loss")
    plt.show()


plot_training(logger)

We see that the algorithm has been getting better and better solutions, and converges to some average loss value. The fact that we see convergence is promising: this suggests the algorithm has reached a (possibly local) minimum. We expect that this corresponds to the BBS algorithm producing bit strings that encode good solutions. We display the best solution that the algorithm has found on the map.

In [ ]:
# Gets the list of paths taken in the solution
roads_taken = []
for i in range(len(path) - 1):
    roads_taken.append([path[i], path[i + 1]])

# Adds the path between the end location and start location
roads_taken.append([path[-1], path[0]])

# Create a figure and axis to plot the image
fig, ax = plt.subplots(figsize=(14, 10))
ax.imshow(img)

# Plot circles at each location
for i in range(len(coordinates)):
    (x, y) = coordinates[i]
    circle = plt.Circle((x, y), radius=10, color="red", fill=False, linewidth=2)
    ax.add_patch(circle)

    # Add the city name at each location
    ax.text(x, y, f"{city_names[i]}", color="black", fontsize=12, ha="center", va="top")

# Set limits to the image dimensions
ax.set_xlim(0, img_width)
ax.set_ylim(img_height, 0)  # Invert y-axis to match image coordinates (top-left origin)

# Hide the axes
ax.axis("off")

x = [a for [a, b] in coordinates]
y = [b for [a, b] in coordinates]

for i in range(len(x)):
    for j in range(i + 1, len(x)):
        if [i, j] in roads_taken or [j, i] in roads_taken:
            plt.plot([x[i], x[j]], [y[i], y[j]], color="red")
        else:
            # plt.plot([x[i], x[j]], [y[i], y[j]], color='gray')
            pass

# Display the plot
plt.show()

And there we have it! Running the algorithm, the salesperson has got a route that they can take which takes them across all of the locations quickly in one round trip. We may not have got to the best possible solution (although for this example we should), but we have got to a good solution and have done so relatively quickly. With there being lots of different possible routes to consider, it would take a very long time to consider all of them to find the perfect solution. Using BBS, the salesperson quickly gets a good solution.

## Random Data Sandbox & Adding Constraints

In this section, we will generate random location data for the travelling salesperson problem, and then will show how to introduce *constraints* into the problem. By constraints, we mean limitations on which solutions we wish to consider. For example, we may not only want to find the shortest route around the points, but also one which visits, say, location 4 directly after location 1. 

First, we generate 7 randomly placed locations and consider running the BBS algorithm for the general travelling salesperson problem as we did before.

In [ ]:
n_locations = 7  # Change the number of locations we're generating


# Generates n_locations random coordinates defining the locations we want to visit
def location_gen(n_locations, x_ax_size=(0, 100), y_ax_size=(0, 100)):
    x = np.random.uniform(x_ax_size[0], x_ax_size[1], n_locations)
    y = np.random.uniform(y_ax_size[0], y_ax_size[1], n_locations)

    return x, y


x, y = location_gen(n_locations=n_locations)

x, y = location_gen(n_locations=n_locations)

# We plot the generated locations
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color="gray", marker="o", s=500)


# Add labels to each point
for i in range(len(x)):
    plt.text(x[i], y[i], f"{i}", fontsize=12, ha="center", va="center", color="black")

plt.axis("off")
plt.show()


# Stores the lengths of each of the roads between locations
road_lengths = [[0 for i in range(n_locations)] for j in range(n_locations)]

for i in range(n_locations):
    for j in range(n_locations):
        # Gets the length of the road between location i and j
        x_length = abs(x[i] - x[j])
        y_length = abs(y[i] - y[j])

        road_length = np.sqrt(x_length**2 + y_length**2)
        road_lengths[i][j] = road_length

We now run the BBS algorithm exactly as we did in the previous examples to find the shortest path around all of the points.

In [ ]:
logger = Logger(log_dir=None)
length_cost_fn = cost_fn_fact(road_lengths, n_locations)

# n bits we need to encode all possible permutations
n_bits = int(np.ceil(np.log2(factorial(n_locations))))

# Initialising the algorithm
bbs = BinaryBosonicSolver(n_bits, length_cost_fn)

# Training the algorithm
bbs.train(learning_rate=1e-2, updates=200, print_frequency=50, logger=logger)
print("Training finished")

opt_sol = np.array(bbs.config_min_encountered)
opt_sol = opt_sol.astype(int)
path = binary_to_permutation("".join(map(str, opt_sol)), n_locations)

roads_taken = []

# Gets the list of paths taken in the solution
for i in range(len(path) - 1):
    roads_taken.append([path[i], path[i + 1]])

# Adds the path between the end location and start location
roads_taken.append([path[-1], path[0]])

In [ ]:
# We plot the optimal path found
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color="gray", marker="o", s=500)

# Draw the roads between the locations
for i in range(len(x)):
    for j in range(i + 1, len(x)):
        if [i, j] in roads_taken or [j, i] in roads_taken:
            plt.plot([x[i], x[j]], [y[i], y[j]], color="red")
        else:
            # plt.plot([x[i], x[j]], [y[i], y[j]], color='gray')
            pass

# Add labels to each point
for i in range(len(x)):
    plt.text(x[i], y[i], f"{i}", fontsize=12, ha="center", va="center", color="black")

plt.axis("off")
plt.show()

Everything is working, and we are finding good optimal paths! Now, we want to introduce constraints. As mentioned before, say we want solutions that visit location 4 directly after location 1. Recall in the introductory `optimisation_intro.ipynb` notebook, for the knapsack problem we wanted to only consider solutions that didn't go over the max weight limit. To achieve this, we penalised any solution that went over the weight capacity. We will proceed in exactly the same way here: we will add a check to our cost function to see whether a solution breaks an imposed constraint and, if it does, we penalise the solution heavily. This will dissuade the algorithm from considering such solutions.

We implement this by defining a function `check_visited_next` that checks if a candidate route (stored in the form of a permutation as previously) visits 4 directly after 1. We then add a check for this in the cost function. If a path doesn't visit 4 after 1, then we artificially increase that paths length (add a penalty).

In [ ]:
# We define a new cost function that adds a penalty term if a path doesn't visit 4 before or after 1

# This function will take as input a permutation (list of location visit order) and check if a location
#  comes directly before or after another
def check_visited_next(perm, first, second):
    first_index = perm.index(first)

    if (
        perm[(first_index + 1) % len(perm)] == second
        or perm[(first_index - 1) % len(perm)] == second
    ):
        return True

    else:
        return False


def Penalty_cost_fn_fact(road_lengths, n_locations, penalty):
    def cost_fn(bit_string):
        bit_string = bit_string.astype(int)

        # Converts the numpy vector into a string
        bit_str = "".join(map(str, bit_string))

        # Converts the bit string into a permutation
        perm = binary_to_permutation(bit_str, n_locations)

        # Gets the length associated with that path
        path_length = path_length_calc(path=perm, road_lengths=road_lengths)

        # NEW ADDITION:
        # If 4 isn't visited before or after 1, we add a big penalty term to the path length returned
        if not check_visited_next(
            perm, 1, 4
        ):  # change the numbers 1 and 4 here to change the two locations we want to be visited one after another
            penalty_term = penalty

            path_length += penalty_term

        return path_length

    return cost_fn

Now we've added this penalty into the cost function the BBS algorithm is minimising. Running the algorithm exactly as before, we will see that this is taken into account by the algorithm, and the solution we find does in fact visit 4 after 1.

In [ ]:
logger = Logger(log_dir=None)

# We change the cost function we are using to the new 'Penalty' cost function
length_cost_fn = Penalty_cost_fn_fact(road_lengths, n_locations, 1000)

# Initialising the algorithm
bbs = BinaryBosonicSolver(
    n_bits, length_cost_fn, tbi_params={"tbi_type": "single-loop"}
)

# Training the algorithm
bbs.train(learning_rate=1e-2, updates=200, print_frequency=50, logger=logger)
print("Training finished")

opt_sol = np.array(bbs.config_min_encountered)
opt_sol = opt_sol.astype(int)
path = binary_to_permutation("".join(map(str, opt_sol)), n_locations)

roads_taken = []

# Gets the list of paths taken in the solution
for i in range(len(path) - 1):
    roads_taken.append([path[i], path[i + 1]])

# Adds the path between the end location and start location
roads_taken.append([path[-1], path[0]])

With training finished, we now output the solution that we got. We see that we get a route that visits location 4 after location 1, while minimising its total distance under this constraint. This is exactly what we hoped to get. Any other constraint can be added in exactly the same manner.

In [ ]:
# We plot the optimal path found
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color="gray", marker="o", s=500)

# Draw the roads between the locations
for i in range(len(x)):
    for j in range(i + 1, len(x)):
        if [i, j] in roads_taken or [j, i] in roads_taken:
            plt.plot([x[i], x[j]], [y[i], y[j]], color="red")
        else:
            # plt.plot([x[i], x[j]], [y[i], y[j]], color='gray')
            pass

# Add labels to each point
for i in range(len(x)):
    plt.text(x[i], y[i], f"{i}", fontsize=12, ha="center", va="center", color="black")

plt.axis("off")
plt.show()

In this notebook we have introduced the travelling salesman problem, seen how it can be posed as a binary optimisation task, and set the BBS algorithm to find good solutions to the problem using both artificial and real-world data. Finally, we created a random data 'sandbox' and considered adding constraints to the problem to reduce the number of solutions that we consider.